`Clickbait Detection Notebook`

Installing necessary packages:

In [2]:
import sys
!{sys.executable} -m pip install -r requirements.txt

  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached daal-2021.5.3-py2.py3-none-manylinux1_x86_64.whl (284.3 MB)
  Using cached daal4py-2021.5.3-py38-none-manylinux1_x86_64.whl (22.6 MB)
  Using cached fonttools-4.33.3-py3-none-any.whl (930 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
  Using cached kiwisolver-1.4.2-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.2 MB)
  Using cached matplotlib-3.5.2-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.3 MB)
  Using cached numpy-1.22.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.9 MB)
  Using cached pandas-1.4.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.7 MB)
  Using cached Pillow-9.1.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached pytz-2022.1-py2.py3-none-any.whl (503 kB)
  Using cached scikit_learn-1.1.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (31.2 MB)
  Using cached scikit_learn_intelex-2021.5.3-py38

<b>Part 1:</b>
<br>This first part runs through training four classifiers on training data. Then, the classifiers are tested on either the training data or external data. The accuracy and f1 scores are found for each classifier.

<br><u>The four classifiers are:</u>
    <br>Multinomial Naive Bayes
    <br>Stochastic Gradient Descent
    <br>Perceptron
    <br>Support Vector Machine

In [4]:
# Part 1

# Import cell
# MAY HAVE TO RUN CELL TWICE IF IT PRODUCES ERRORS

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn import svm

from sklearn import metrics

import matplotlib.pyplot as plt


# Used to increase parallel computing on CPU
# pip install scikit-learn-intelex
# Uncomment `from sklearnex import patch_sklearn` to import the patch
#from sklearnex import patch_sklearn

# Uncomment `patch_sklearn()` function in to enable the patch
# Doesn't work all the time, so that's why it's commented out
#patch_sklearn()

In [ ]:
# Part 1

# Reading data into pandas dataframe

# Defining Constants
clickbait_title_column = "headline"
is_clickbait_column = "clickbait"

# Pandas dataframe df
df = pd.read_csv("input_data/clickbait_consensus.csv")

y = df[is_clickbait_column]
X = df[clickbait_title_column]

print("Data formated")

In [ ]:
# Part 1

# Vectorizing data in order for classifiers to use them
# This is required because classifiers need to numerical data, 
#   so the titles (wihch are strings) need to be transformed numerically

vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(X)

print("Data vectorized")

In [ ]:
# Part 1

# Asking user what data to test classifiers on

print("-------------------------TEST DATA-----------------------------------------\n")

which_data = input("Would you like to test on training data (enter 0) or pre-determined, external data (enter 1)? ")

if (which_data == "0"):
    print("Training data chosen")
    test_multi_titles = df[clickbait_title_column]
    test_multi_titles_nparray = df[is_clickbait_column].to_numpy()

    vectors_test = vectorizer.transform(test_multi_titles)

else:
    print("External data chosen")
    
    test_df = pd.read_csv("input_data/clickbait_ratio_flattened.csv")

    test_multi_titles = test_df[clickbait_title_column]
    test_multi_titles_nparray = test_df[is_clickbait_column].to_numpy()

    vectors_test = vectorizer.transform(test_multi_titles)

In [ ]:
# Part 1

# Multinomial Naive Bayes Classification

print("-------------------------MULTINOMIALNB PREDICTING-----------------------------------------\n")

multiNB_clf = MultinomialNB(alpha=0.00001)
multiNB_clf.fit(vectors, y)


multiNB_pred = multiNB_clf.predict(vectors_test)

print("Prediction:", multiNB_pred)
print()

acc_score_multiNB = metrics.accuracy_score(test_multi_titles_nparray, multiNB_pred)
f1_score_multNB = metrics.f1_score(test_multi_titles_nparray, multiNB_pred, average='macro')
roc_score_multiNB = metrics.roc_auc_score(test_multi_titles_nparray, multiNB_pred)

print('Total accuracy classification score: {}'.format(acc_score_multiNB))
print('Total F1 classification score: {}'.format(f1_score_multNB))
print('Total ROC classification score: {}'.format(roc_score_multiNB))

# precision_score_multiNB = metrics.precision_score(test_multi_titles_nparray, multiNB_pred, average='macro')
# recall_score_multiNB = metrics.recall_score(test_multi_titles_nparray, multiNB_pred, average='macro')

# print('\nTotal precision score: {}'.format(precision_score_multiNB))
# print('Total recall score: {}'.format(recall_score_multiNB))

# Get data for ROC Curve
multiNB_fpr, multiNB_tpr, thresholds = metrics.roc_curve(test_multi_titles_nparray, multiNB_pred)

# Create ROC curve
plt.plot(multiNB_fpr, multiNB_tpr)

# Add axis labels to plot
plt.title('Multinomial Naive Bayes ROC Curve')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

# Display plot and save plot as jpg
#plt.show()
plt.savefig("mulitNB_roc_curve.jpg")


In [ ]:
# Part 1

# Stochastic Gradient Descent Classification

print("-------------------------STOCHSTIC GRADIENT DESCENT (SGD) PREDICTING---------------------------------------\n")

SGD_clf = SGDClassifier(loss="huber", penalty="l2", max_iter=5000)
SGD_clf.fit(vectors, y)

SGD_pred = SGD_clf.predict(vectors_test)

print("Prediction:", SGD_pred)
print()

acc_score_SGD = metrics.accuracy_score(test_multi_titles_nparray, SGD_pred)
f1_score_SGD = metrics.f1_score(test_multi_titles_nparray, SGD_pred, average='macro')
roc_score_SGD = metrics.roc_auc_score(test_multi_titles_nparray, SGD_pred)

print('Total accuracy classification score: {}'.format(acc_score_SGD))
print('Total F1 classification score: {}'.format(f1_score_SGD))
print('Total ROC classification score: {}'.format(roc_score_SGD))


# Get data for ROC Curve
SGD_fpr, SGD_tpr, thresholds = metrics.roc_curve(test_multi_titles_nparray, multiNB_pred)

# Create ROC curve
plt.plot(SGD_fpr, SGD_tpr)

# Add axis labels to plot
plt.title('Stochastic Gradient Descent ROC Curve')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

# Display plot and save plot as jpg
#plt.show()
plt.savefig("SGD_roc_curve.jpg")

In [ ]:
# Part 1

# Perceptron Classification

print("-------------------------PERCEPTRON PREDICTING---------------------------------------\n")

perceptron_clf = Perceptron(tol=1e-3, random_state=0)
perceptron_clf.fit(vectors, y)

perceptron_pred = perceptron_clf.predict(vectors_test)

print("Prediction:", perceptron_pred)
print()

acc_score_perceptron = metrics.accuracy_score(test_multi_titles_nparray, perceptron_pred)
f1_score_perceptron = metrics.f1_score(test_multi_titles_nparray, perceptron_pred, average='macro')
roc_score_perceptron = metrics.roc_auc_score(test_multi_titles_nparray, perceptron_pred)

print('Total accuracy classification score: {}'.format(acc_score_perceptron))
print('Total F1 classification score: {}'.format(f1_score_perceptron))
print('Total ROC classification score: {}'.format(roc_score_perceptron))


# Get data for ROC Curve
perceptron_fpr, perceptron_tpr, thresholds = metrics.roc_curve(test_multi_titles_nparray, multiNB_pred)

# Create ROC curve
plt.plot(perceptron_fpr, perceptron_tpr)

# Add axis labels to plot
plt.title('Perceptron ROC Curve')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

# Display plot and save plot as jpg
#plt.show()
plt.savefig("perceptron_roc_curve.jpg")

In [ ]:
# Part 1

# Support Vector Machine Classification

print("-------------------------SVM PREDICTING---------------------------------------\n")

SVM_clf = svm.SVC(gamma=3, kernel='sigmoid')
SVM_clf.fit(vectors, y)

SVM_pred = SVM_clf.predict(vectors_test)

print("Prediction:", SVM_pred)
print()

acc_score_SVM = metrics.accuracy_score(test_multi_titles_nparray, SVM_pred)
f1_score_SVM = metrics.f1_score(test_multi_titles_nparray, SVM_pred, average='macro')
roc_score_SVM = metrics.roc_auc_score(test_multi_titles_nparray, SVM_pred)

print('Total accuracy classification score: {}'.format(acc_score_SVM))
print('Total F1 classification score: {}'.format(f1_score_SVM))
print('Total ROC classification score: {}'.format(roc_score_SVM))


# Get data for ROC Curve
SVM_fpr, SVM_tpr, thresholds = metrics.roc_curve(test_multi_titles_nparray, multiNB_pred)

# Create ROC curve
plt.plot(SVM_fpr, SVM_tpr)

# Add axis labels to plot
plt.title('Support Vector Machine ROC Curve')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

# Display plot and save plot as jpg
#plt.show()
plt.savefig("SVM_roc_curve.jpg")

<b>Part 2</b>
<br> Part 2 allows you, the user, to enter in your own titles that you find on the internet or create on your own and run it throught the four classifiers to see if it's clickbait.
<br>
<br> The notebook will walk through the functions for training the classifiers (same process as above, only it is formatted professionally now). Then, you will be prompted to enter text so that the classifiers can predict clickbait status.

In [ ]:
# Part 2

# trainClassifiers() runs a compiled .csv file through all four classifiers of over 30,000 data points.

def trainClassifiers():

    print("\nTraining classifiers...", end="")

     # Defining Constants
    clickbait_title_column = "headline"
    is_clickbait_column = "clickbait"

    # Pandas dataframe df
    df = pd.read_csv("input_data/clickbait_compilation.csv")


    y = df[is_clickbait_column]
    X = df[clickbait_title_column].str.lower() # All titles are in lowercase


    vectorizer = CountVectorizer()
    vectors = vectorizer.fit_transform(X)


    # MultinomialNB
    multiNB_clf = MultinomialNB(alpha=0.00001)
    multiNB_clf.fit(vectors, y)

    # SGD
    SGD_clf = SGDClassifier(loss="huber", penalty="l2", max_iter=5000)
    SGD_clf.fit(vectors, y)

    # Perceptron
    perceptron_clf = Perceptron(tol=1e-3, random_state=0)
    perceptron_clf.fit(vectors, y)

    # SVM
    SVM_clf = svm.SVC(gamma=3, kernel='sigmoid')
    SVM_clf.fit(vectors, y)

    print("done")

    return vectorizer, multiNB_clf, SGD_clf, perceptron_clf, SVM_clf

In [ ]:
# Part 2

# predictWithClassifiers() takes the user's entry and predicts clickbait status on all four classifiers

def predictWithClassifiers(user_clickbait_title, vectorizer, multiNB_pred, SGD_pred, perceptron_pred, SVM_pred):
        
        print("\n-----------------------------------------------------------")
        print("STATISTICS:")

        # Vectorize the title to pass into the prediction of the classifier
        vector_user_title_test = vectorizer.transform(user_clickbait_title)

        # Multinomial Prediction
        multiNB_pred = multiNB_clf.predict(vector_user_title_test)
        print("Multinomial Naive Bayes Prediction:\t\t", multiNB_pred)


        # Stochastic Gradient Descent Prediction
        SGD_pred = SGD_clf.predict(vector_user_title_test)
        print("Stochastic Gradient Descent Prediction:\t\t", SGD_pred)


        # Perceptron Prediction
        perceptron_pred = perceptron_clf.predict(vector_user_title_test)
        print("Perceptron Prediction:\t\t\t\t", perceptron_pred)
        

        # SVM prediction
        SVM_pred = SVM_clf.predict(vector_user_title_test)
        print("Support Vector Machine Prediction:\t\t", SVM_pred)

        return multiNB_pred, SGD_pred, perceptron_pred, SVM_pred

In [ ]:
# Part 2

# classifierClickbaitStatus()) finds the clickbait status of all four classifiers by averaging their predictions
# Prediction values are [0] for a non-clickbait title and [1] for a clickbait title

def classifierClickbaitRatio(multiNB_pred, SGD_pred, perceptron_pred, SVM_pred):
    classifier_clickbait_ratio = (multiNB_pred+SGD_pred+perceptron_pred+SVM_pred)/ num_of_classifiers

    print("\nTotal Classification Clickbait Status:\t\t\t", classifier_clickbait_ratio)
    print("\n-----------------------------------------------------------")

    return classifier_clickbait_ratio


In [ ]:
# Part 2

# determineClickbait() determines if the title is clickbait based on the four classifier predictions

def determineClickbait(classifier_clickbait_ratio, isClickbait_threshold, possiblyClickbait_threshold):
    # If the classifier accuracy is above the threshold, title is clickbait
    # Else, it's below the threshold, so title is not clickbait
    
    if (classifier_clickbait_ratio >= isClickbait_threshold):
        print("\nCLICKBAIT")
    elif (classifier_clickbait_ratio == possiblyClickbait_threshold):
        print("\nPOSSIBLY CLICKBAIT")
    else:
        print("\nNOT CLICKBAIT")

In [ ]:
# Part 2

# This cell trains the classifiers on the training data

# train Classifiers on over 30,000 data points
vectorizer, multiNB_clf, SGD_clf, perceptron_clf, SVM_clf = trainClassifiers()

In [ ]:
# Part 2

# This cell asks for user input and runs prediction on the input
# Displays results of prediction as well as clickbait determination

# Constants
num_of_classifiers = 4
isClickbait_threshold = 0.75
possiblyClickbait_threshold = 0.5

# Get user clickbait title
user_clickbait_title = input("\n\nEnter a title to see if it's clickbait (-1 to quit):\n").lower()

if (user_clickbait_title != "-1"):

    # Turn string title into list iterable for vectorization
    user_clickbait_title = [user_clickbait_title]

    # Predicting the clickbait classification on all classifiers
    multiNB_pred, SGD_pred, perceptron_pred, SVM_pred = predictWithClassifiers(user_clickbait_title, vectorizer, multiNB_clf, SGD_clf, perceptron_clf, SVM_clf)

    # Total mean classifier accuracy to determine if the title is clickbait
    classifier_clickbait_ratio = classifierClickbaitRatio(multiNB_pred, SGD_pred, perceptron_pred, SVM_pred)

    # Determing that clickbait status of the user's title
    determineClickbait(classifier_clickbait_ratio, isClickbait_threshold, possiblyClickbait_threshold)

else:
    print("Bye now!")